# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will also be using the watchlist you just created to figure out which streaming services contain the shows that you want to watch next so you can decide which one is worth the money to you.

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. Remember that during the prep work, we learned that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell.

Watchlist: 1) Community, 2) Star Wars: The Clone Wars, 3) The Mandalorian, 4) Star Wars Rebels 5) Cobra Kai, 6) Bridgerton, 7) The Circle, 8) The Toys that Made Us, 9) Saturday Night Live, 10) One Tree Hill

## Database Setup

To start with, import the necessary libraries and create a dataframe from the provided CSV. Print the info out for the dataframe. After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces in the names or unusual characters such as `"+"`. Print out the info for the dataframe again to ensure that your changes were effective.

In [13]:
import pandas as pd
import sqlite3 as sl

tvdf=pd.read_csv('~/Library/Mobile Documents/com~apple~CloudDocs/LC_DA/Git/databases-and-py-studio/tv_shows.csv')
tvdf.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


In [14]:
tvdf = tvdf.drop(columns='Unnamed: 0')
tvdf.columns

Index(['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'Type'],
      dtype='object')

With your dataframe at the ready, create a new database called `tv`. Add a new table to your database called `shows` using the data in the dataframe. 

In [15]:
con = sl.connect('tv.db')

tvdf.to_sql('shows', con, if_exists='replace')

5368

With your new table and database set up, print out the top 20 records in the `shows` table.

In [16]:
with con:
    data = con.execute("SELECT * FROM shows LIMIT 20")
for row in data:
    print(row)

(0, 1, 'Breaking Bad', 2008, '18+', '9.4/10', '100/100', 1, 0, 0, 0, 1)
(1, 2, 'Stranger Things', 2016, '16+', '8.7/10', '96/100', 1, 0, 0, 0, 1)
(2, 3, 'Attack on Titan', 2013, '18+', '9.0/10', '95/100', 1, 1, 0, 0, 1)
(3, 4, 'Better Call Saul', 2015, '18+', '8.8/10', '94/100', 1, 0, 0, 0, 1)
(4, 5, 'Dark', 2017, '16+', '8.8/10', '93/100', 1, 0, 0, 0, 1)
(5, 6, 'Avatar: The Last Airbender', 2005, '7+', '9.3/10', '93/100', 1, 0, 1, 0, 1)
(6, 7, 'Peaky Blinders', 2013, '18+', '8.8/10', '93/100', 1, 0, 0, 0, 1)
(7, 8, 'The Walking Dead', 2010, '18+', '8.2/10', '93/100', 1, 0, 0, 0, 1)
(8, 9, 'Black Mirror', 2011, '18+', '8.8/10', '92/100', 1, 0, 0, 0, 1)
(9, 10, "The Queen's Gambit", 2020, '18+', '8.6/10', '92/100', 1, 0, 0, 0, 1)
(10, 11, 'Mindhunter', 2017, '18+', '8.6/10', '90/100', 1, 0, 0, 0, 1)
(11, 12, 'Community', 2009, '7+', '8.5/10', '90/100', 1, 1, 1, 0, 1)
(12, 13, 'Narcos', 2015, '18+', '8.8/10', '90/100', 1, 0, 0, 0, 1)
(13, 14, 'Shameless', 2011, '18+', '8.5/10', '90/100',

Now add a table that includes an id number and the titles on your watchlist.

In [17]:
with con:
    con.execute('''
        CREATE TABLE watchlist(
            ID int PRIMARY KEY NOT NULL,
            title nvarchar(50) NOT NULL
            )
                ''')
con.commit()

sql = 'INSERT INTO watchlist (ID, title) values(?, ?)'
data = [
    (1, 'Community'), 
    (2, 'Star Wars: The Clone Wars'), 
    (3, 'The Mandalorian'),
    (4, 'Star Wars Rebels'),
    (5, 'Cobra Kai'),
    (6, 'Bridgerton'),
    (7, 'The Circle'),
    (8, 'The Toys that Made Us'),
    (9, 'Saturday Night Live'),
    (10, 'One Tree Hill')
]

with con: con.executemany(sql, data)

## Working with the Data

Using either Pandas or SQL, determine how many shows are on each streaming service and what the mean is for each streaming service. 

**Note**: You may notice that the mean represents the percentage of shows in the dataframe that are on each streaming service. Take a moment to think about why that is.

In [18]:
#Agg count function

service_dict=({'Netflix': ['sum','mean'], 'Hulu': ['sum','mean'], 'Prime Video': ['sum','mean'], 'Disney+': ['sum','mean']})
dictionary_agg=tvdf.agg(service_dict)
round(dictionary_agg, 2)

,Netflix,Hulu,Prime Video,Disney+
sum,1971.00,1621.0,1831.00,351.00
mean,0.37,0.3,0.34,0.07


Join your watchlist data with the shows data to determine which streaming services your watchlist shows are on and make a new table in your database using the joined data. Print out the data in your joined table to see what shows on your watchlist on in the original dataset. With the joined data, determine the percentage of your watchlist shows that are on each streaming service and how many of your watchlist shows are on each streaming service.

In [31]:
data = pd.read_sql_query("""
    SELECT *
    FROM shows
    INNER JOIN watchlist ON shows.Title = watchlist.Title """, con)
data.head(20)

,index,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,ID,title
0,11,12,Community,2009,7+,8.5/10,90/100,1,1,1,0,1,1,Community
1,21,22,Cobra Kai,2018,16+,8.6/10,89/100,1,0,0,0,1,5,Cobra Kai
2,120,121,Bridgerton,2020,None,7.3/10,79/100,1,0,0,0,1,6,Bridgerton
3,282,284,The Circle,2020,18+,7.3/10,70/100,1,0,1,0,1,7,The Circle
4,2101,2138,Saturday Night Live,1975,16+,8.0/10,75/100,0,1,0,0,1,9,Saturday Night Live
5,2123,2161,One Tree Hill,2003,16+,7.7/10,74/100,0,1,0,0,1,10,One Tree Hill
6,5058,5377,The Mandalorian,2019,16+,8.8/10,94/100,0,0,0,1,1,3,The Mandalorian
7,5061,5382,Star Wars: The Clone Wars,2008,7+,8.3/10,85/100,0,0,0,1,1,2,Star Wars: The Clone Wars
8,5065,5387,Star Wars Rebels,2014,7+,8.0/10,79/100,0,0,0,1,1,4,Star Wars Rebels


In [33]:
data.head(20).aggregate('sum')

/var/folders/nc/sfbwykc541d4ktbn0q7gz6080000gp/T/ipykernel_33848/1988727613.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.head(20).aggregate('sum')


index                                                          19842
ID                                                             20884
Title              CommunityCobra KaiBridgertonThe CircleSaturday...
Year                                                           18086
IMDb               8.5/108.6/107.3/107.3/108.0/107.7/108.8/108.3/...
Rotten Tomatoes    90/10089/10079/10070/10075/10074/10094/10085/1...
Netflix                                                            4
Hulu                                                               3
Prime Video                                                        2
Disney+                                                            3
Type                                                               9
ID                                                                47
title              CommunityCobra KaiBridgertonThe CircleSaturday...
dtype: object

In [32]:
con.close()

## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present? Yes! If it wasn't present, then it must have been on a service not listed, like Max, but all of my shows were there. 
2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis? Yes, if I had not included all columns, I may have missed that show.
3. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least? Netflix
4. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you? I suppose Netflix would give me the most options, but I would choose Disney+ because it is the only streaming service that has Star Wars.